In [2]:
# Import necessary modules
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import random
import numpy as np

# BB84 Protocol with Eavesdropping
This notebook demonstrates the BB84 protocol with eavesdropping. 
 
The protocol is secure against eavesdropping, as it uses the principles of quantum mechanics to detect any eavesdropping attempts. 

In [3]:
# Setting up the environment
n = 28 # Length of the key to be generated
# Alice's operations

circuit = QuantumCircuit(n, n)


alice_key = np.random.randint(0, 2, size = n)   
alice_basis = np.random.randint(0, 2, size=n)  


for i, bit in enumerate(alice_key): 
    if bit == 1:        
        circuit.x(i)  


for i, basis in enumerate(alice_basis):
    if basis == 1:
        circuit.h(i)  

print(alice_key)

[0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1]


In [4]:
# Eve's operations

eve_basis = np.random.randint(0, 2, size=n) 



for i, basis in enumerate(eve_basis):
    if basis == 1:
        circuit.h(i)  # Apply Hadamard to measure in the X-basis

circuit.measure(range(n),range(n)) #This represents the interception by Eve


simulator = Aer.get_backend('qasm_simulator')
circ = transpile(circuit, simulator)
result = simulator.run(circ, shots = 1).result()
eve_results = result.get_counts(circ).keys()
eve_results = np.array([int(k) for k in list(eve_results)[0]])
eve_results = np.flip(eve_results)
print(eve_results)

eve_circuit = QuantumCircuit(n, n)  # Eve new qubits to send to Bob, trying to emulate Alice's qubits
for i in range(n):

    if eve_results[i] == 1:
        eve_circuit.x(i)  

    if eve_basis[i] == 1:
        eve_circuit.h(i)  

[0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1]


In [5]:
# Bob's operations

bob_basis = np.random.randint(0, 2, size=n)

for i, basis in enumerate(bob_basis):
    if basis == 1:
        eve_circuit.h(i)  # Apply Hadamard for diagonal basis


eve_circuit.measure(range(n), range(n))


simulator = Aer.get_backend('qasm_simulator')
circ = transpile(eve_circuit, simulator)
result = simulator.run(circ, shots = 1).result()
counts = result.get_counts(circ)
#plot_histogram(counts)

In [6]:

total_measurements = sum(counts.values())
print("Total Measurements:", total_measurements)

most_common_result = max(counts, key=counts.get) 
print("Most common result:", most_common_result)

bob_results = most_common_result 
bob_results = np.array([int(k) for k in bob_results]) 
bob_results = np.flip(bob_results)

print("Alice's key:   ", alice_key)
print("Eve's results: ", eve_results)
print("Bob's results: ", bob_results)

Total Measurements: 1
Most common result: 0100101000111101010101100000
Alice's key:    [0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1]
Eve's results:  [0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1]
Bob's results:  [0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0]


In [7]:
# Key generation
print("BB84 Protocol with Eavesdropping, keys should match if there is no eavesdropping")
print("-----------------------------------------------------------------------------")

print("Alice's basis:", alice_basis)
print("Bob's basis:  ", bob_basis)
matching_bases = [i for i in range(n) if alice_basis[i] == bob_basis[i]] # Communication only happens regarding the bases, not the qubits


bob_key = [bob_results[i] for i in matching_bases]
alice_key_shared = [alice_key[i] for i in matching_bases]

print("---------------------------------------------")

print("Alice's key: ", alice_key_shared)
print("Bob's key:   ", bob_key)

BB84 Protocol with Eavesdropping, keys should match if there is no eavesdropping
-----------------------------------------------------------------------------
Alice's basis: [0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1]
Bob's basis:   [1 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 0 0]
---------------------------------------------
Alice's key:  [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1]
Bob's key:    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1]


In [8]:
# Check for eavesdropping

print("Checking for eavesdropping...")
print("Check done by comparing the two complete keys")
print("---------------------------------------------")


if alice_key_shared == bob_key:
    print("No eavesdropping detected. Secure key exchange.")
else:
    print("Eavesdropping detected. Secure key exchange not possible.")
print("---------------------------------------------")

Checking for eavesdropping...
Check done by comparing the two complete keys
---------------------------------------------
Eavesdropping detected. Secure key exchange not possible.
---------------------------------------------


In [9]:

sample_size = len(matching_bases) // 2
print("Sample size:", sample_size)
random_sample = random.sample(range(len(matching_bases)), sample_size)


bob_sample_key = [bob_key[i] for i in random_sample]
alice_sample_key = [alice_key_shared[i] for i in random_sample]

print("Checking for eavesdropping...")
print("Check done by comparing a random sample of the keys")
print("---------------------------------------------------")
print("Alice's sample key: ", alice_sample_key)
print("Bob's sample key:   ", bob_sample_key)
print("---------------------------------------------------")

if alice_sample_key == bob_sample_key:
    print("No eavesdropping detected in random sample.")
else:
    print("Eavesdropping detected in random sample. Secure key exchange not possible.")
    

probability = 0.75 ** sample_size

print("Probability of NOT detecting eavesdropping if it exists:", probability)

print("---------------------------------------------------")

Sample size: 7
Checking for eavesdropping...
Check done by comparing a random sample of the keys
---------------------------------------------------
Alice's sample key:  [1, 1, 0, 1, 0, 0, 1]
Bob's sample key:    [1, 0, 0, 1, 1, 0, 1]
---------------------------------------------------
Eavesdropping detected in random sample. Secure key exchange not possible.
Probability of NOT detecting eavesdropping if it exists: 0.13348388671875
---------------------------------------------------


In [10]:
# Computing the error rate
error_rate = sum([1 for i in range(sample_size) if alice_sample_key[i] != bob_sample_key[i]]) / sample_size
print("Error rate:", error_rate)


Error rate: 0.2857142857142857
